# Purpose

This notebook tracks development of a wrapper for the kameleon software suite to support legacy ccmc models until python 3 model readers become available. 

This notebook should be executed from a python3 environment with kamodo installed.

In [2]:
!which python

/Users/apembrok/miniconda3/envs/kamodo/bin/python


In [3]:
!python --version

Python 3.7.3


In [62]:
from kamodo import kamodofy

In [45]:
from kamodo import Kamodo, kamodofy
from kameleon import kameleon_gateway

import execnet

class Kameleon(Kamodo):
    def __init__(self, filename, python_path, *variables, **kwargs):
        
        self._current_file = filename
        self._python_path = python_path
        self.make_gateway()
        
        self.open_file(filename, *variables)
        
        super(Kameleon, self).__init__(**kwargs)
        
        self.make_interpolators()
        
        
    def get_coords(self):
        """Extracts coordinate list from grid_system_1 metadata:
            STRING: grid_system_1: [X,Y,Z]
        '"""
        coords = self._metadata['grid_system_1'].split(':')[-1].strip()
        coords = ''.join(c for c in coords if c not in '[]') # remove brackets
        coords = [c.strip().lower() for c in coords.split(',')]
        return coords
        
    def make_gateway(self):
        """Create a gateway into python installed with kameleon"""
        gateway_spec = 'popen//dont_write_bytecode//python={}'.format(self._python_path)
        self._gw = execnet.makegateway(gateway_spec)
        self._ch = self._gw.remote_exec(kameleon_gateway)
        
    def open_file(self, file_name, *variables):
        args = [file_name] + list(variables)
        cmd_args = ", ".join(["'{}'".format(s) for s in args])
        cmd = "initialize({})".format(cmd_args)
        try:
            self._ch.send(cmd) # execute func-call remotely
        except self._ch.RemoteError as m:
            print(m)
            
        try:
            self._metadata = self._ch.receive()
            for variable_name in variables:
                assert variable_name in self._metadata['variables']
            # sort variable metadata in the same order as input
            self._metadata['variables'] = {variable : self._metadata['variables'][variable] \
                                           for variable in variables}
            
            self._current_file = file_name
        except:
            print('could not open file')
            print('offending command:\n\t{}'.format(cmd))
            print(self._metadata)
        print('{} opened'.format(file_name))
        
        
    def interpolate(self, variable_name, c0, c1, c2):

        self._ch.send("interpolate('{}',{},{},{})".format(variable_name, c0, c1, c2))

        return self._ch.receive()

    def make_interpolators(self):
        for variable, metadata in self._metadata['variables'].items():
            coords = self.get_coords()
            units = metadata['units']
            exec_str = """def interpolate({components}):
                package = '{varname}', {c0}, {c1}, {c2}
                self._ch.send(package)
                return self._ch.receive()
            """.format(components = ', '.join(coords),
                      varname = variable,
                      c0 = coords[0],
                      c1 = coords[1],
                      c2 = coords[2])
            d = {'self':self}
            exec(exec_str, d)
            interp_func = d['interpolate']
            self[variable] = kamodofy(interp_func, units = units)

# may need to specify the full path to lfm, else file doesn't load correctly
fname = '/Users/apembrok/Work/Kameleon_Sample_Data/LFM/Nov2015.LFM.S1-OF_mhd_2002-09-30T03-16-00Z.cdf'
python_path = '/Users/apembrok/miniconda2/envs/klive/bin/python'
kameleon = Kameleon(fname, python_path, 'rho', 'p', 'bx','by','bz')
        

/Users/apembrok/Work/Kameleon_Sample_Data/LFM/Nov2015.LFM.S1-OF_mhd_2002-09-30T03-16-00Z.cdf opened


In [46]:
kameleon

Kameleon([(rho(x, y, z), <function interpolate(x, y, z)>),
          (rho, <function interpolate(x, y, z)>),
          (p(x, y, z), <function interpolate(x, y, z)>),
          (p, <function interpolate(x, y, z)>),
          (bx(x, y, z), <function interpolate(x, y, z)>),
          (bx, <function interpolate(x, y, z)>),
          (by(x, y, z), <function interpolate(x, y, z)>),
          (by, <function interpolate(x, y, z)>),
          (bz(x, y, z), <function interpolate(x, y, z)>),
          (bz, <function interpolate(x, y, z)>)])

In [47]:
kameleon.p(-30,0,0)

0.005562467034906149

In [48]:
kameleon.rho(-30,0,0)

0.3747256398200989

In [49]:
kameleon.bx(-30, 0, 0)

-14.527422904968262

In [1]:
%load_ext autoreload

%autoreload 2